In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import pathlib

In [3]:
python_path = pathlib.Path('.').absolute().parent/'python'
os.sys.path.insert(1, str(python_path))

In [4]:
import pinocchio as pin
from robot_properties_kuka.config import IiwaConfig

import meshcat
import meshcat.transformations as tf
import meshcat.geometry as g

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from tqdm import trange
import wandb

In [5]:
from vocam.qpnet import DataUtils, QPNet
import vocam.qpnet
from vocam.inverse_qp import IOC

In [6]:
robot = IiwaConfig.buildRobotWrapper()
model, data = robot.model, robot.data
f_id = model.getFrameId("EE")

viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=False)
viz.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/


In [7]:
run = wandb.init(project="vocam", group="kuka_qpnet_eval", entity="hjzhu")

wandb: Currently logged in as: hjzhu. Use `wandb login --relogin` to force relogin


In [8]:
wandb.config.nq = model.nq
wandb.config.nv = model.nv

wandb.config.n_col = 5
wandb.config.u_max = [2.5, 2.5, 2.5, 1.5, 1.5, 1.5, 1.0]
wandb.config.dt = 0.05

wandb.config.n_vars = 3 * model.nq * wandb.config.n_col + 2 * model.nq
wandb.config.isvec = True
wandb.config.lr_qp = 1e-1
wandb.config.max_it = 100
wandb.config.task_horizon = 30

wandb.config.input_size = model.nq + model.nv + 3
wandb.config.output_size = 2 * wandb.config.n_vars

In [9]:
data_artifacts = run.use_artifact('hjzhu/vocam/qpnet_datasets:latest', type='dataset')
data_dir = data_artifacts.download()
data_test = torch.load(data_dir + "/data_20_40.pt")
unzipped = list(zip(*data_test))
x_train = torch.vstack([*unzipped[0]]).to('cpu')
y_train = torch.vstack([*unzipped[1]]).to('cpu')

In [10]:
network = QPNet(wandb.config.input_size, 
                wandb.config.output_size).to('cpu')
model_artifacts = run.use_artifact('hjzhu/vocam/qpnet_models:latest', type='model')
model_dir = model_artifacts.download()
network.load(model_dir + "/qpnet_41.pt")
network.eval()

QPNet(
  (swish): SiLU()
  (fc1): Linear(in_features=17, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=512, out_features=1024, bias=True)
  (bn2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (bn3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (out): Linear(in_features=512, out_features=238, bias=True)
)

In [11]:
nq = wandb.config.nq
nv = wandb.config.nv
n_col = wandb.config.n_col
u_max = wandb.config.u_max
dt = wandb.config.dt
isvec = wandb.config.isvec
lr_qp = wandb.config.lr_qp
n_vars = wandb.config.n_vars

In [20]:
# visualize the init configuration and the desired location for a random data point
k = 150
x_in = x_train[k].clone().detach()
y = y_train[k].clone().detach()

q_init = x_in[:nq].numpy()
x_des = x_in[-3:]

print(k)
print(x_des)

viz.viewer["box"].set_object(g.Sphere(0.05), 
                         g.MeshLambertMaterial(
                             color=0xff22dd,
                             reflectivity=0.8))
viz.viewer["box"].set_transform(tf.translation_matrix(x_des[-3:].detach().numpy()))
viz.display(q_init)

150
tensor([0.4058, 0.5138, 0.2468])


In [21]:
ddq = []
for j in range(50):
    ioc = IOC(n_col, nq, u_max, dt, eps = 1.0, isvec=isvec)
    optimizer = torch.optim.Adam(ioc.parameters(), lr=lr_qp)
    pred = network(x_in[None,:]).squeeze()
    
    if not isvec:
        ioc.weight = torch.nn.Parameter(torch.reshape(pred[0:n_vars**2], (n_vars, n_vars)))
        ioc.x_nom = torch.nn.Parameter(pred[n_vars**2:])
    else:
        ioc.weight = torch.nn.Parameter(pred[0:n_vars])
        ioc.x_nom = torch.nn.Parameter(pred[n_vars:])
        
    x_pred = ioc((x_in[:-3]).detach().numpy()) 
    x_pred = x_pred.detach().numpy()
    ddq.append(x_pred[2*nq:3*nq])

    for i in range(n_col+1):
        q = x_pred[3*nq*i:3*nq*i + nq]
        dq = x_pred[3*nq*i + nq:3*nq*i + 2*nq]

        pin.forwardKinematics(model, data, q, dq, np.zeros(nv))
        pin.updateFramePlacements(model, data)

        viz.display(q)
        time.sleep(0.05)
    
    x_in[0:2*nq] = torch.tensor(x_pred[-2*nq:])
    
ddq = np.vstack(ddq)

In [ ]:
x_train.